This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'LxC7H9m2oPCaULoENqdp'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
from collections import namedtuple
from pathlib import Path

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
res = requests.get(f"https://quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}&start_date=2020-05-20&end_date=2020-05-20")
print(res.text)

{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-05-20","end_date":"2020-05-20","frequency":"daily","data":[["2020-05-20",92.5,93.3,91.3,91.8,null,169820.0,15657134.85,null,null,null]],"collapse":null,"order":null}}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json.loads(res.text)

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-05-20',
  'end_date': '2020-05-20',
  'frequency': 'daily',
  'data': [['2020-05-20',
    92.5,
    93.3,
    91.3,
    91.8,
    None,
    169820.0,
    15657134.85,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
def get_quandl_data(api_key, data_source, ticker, start_date, end_date, use_cache=None):
    """
    Retrieves daily stock data from Quandl API (v3).

    Parameters
    ----------
    api_key: str
        Provided key from Quandl after registering for an account
    data_source: str
        Indicates which stock exchange the data should be returned from
    ticker: str
        The company ticker for the data_source provided
    start_date: str
        The first date (inclusive) in the date range requested. This
        should be formatted as 'YYYY-MM-DD'.
    end_date: str
        The last date (inclusive) in the date range requested. This
        should be formatted as 'YYYY-MM-DD'.
    use_cache: str
        Path to a locally saved copy in case API calls are running low.
        Must be a properly formatted local path.
    
    Returns
    -------
    str
        A string with a properly formatted JSON payload featuring,
        among other things, an array called 'column_names' and an
        object called 'data' with a nested array for each date in
        the provided range.
    """
    if use_cache:
        try:
            with open(use_cache, 'r') as infile:
                return json.load(infile)
        except:
            print("File does not exist")
            return None
    BASE_URL = f"https://quandl.com/api/v3/datasets/{data_source}/{ticker}/data.json"
    res = requests.get(f"{BASE_URL}?api_key={api_key}&start_date={start_date}&end_date={end_date}")
    return json.loads(res.text)
    

def create_daily_dict(data):
    """
    Takes a dictionary from a JSON payload in a set format as returned by
    Quandl (now NASDAQ Data) and turns it into a dictionary with dates as
    the key and the daily metrics as a named tuple.
    
    Parameters
    ----------
    data: dict
        Properly formatted dictionary matching the Quandl JSON payload.
        The function assumes that this dict is formatted as expected,
        the function will throw an error if not.
    
    Returns
    -------
    dict
        Dictionary with each date as the key (YYYY-MM-DD) and a
        named tuple as the value using the column_names returned
        in the JSON payload.
    """
    cols = [x.replace(' ', '_').lower() for x in data['dataset_data']['column_names'][1:]]
    FSE_Date = namedtuple('FSE_Date', cols)
    daily_dict = {x[0]: FSE_Date(*x[1:]) for x in data['dataset_data']['data']}
    return {k: daily_dict[k] for k in sorted(daily_dict)}


In [6]:
# Request for FSE ticker AFX_X for the entire year of 2017
res = get_quandl_data(API_KEY, 'FSE', 'AFX_X', '2017-01-01', '2017-12-31')
dd = create_daily_dict(res)
# Well shoot. No weekends means that I won't be able to just use date as
# an index like I was planning. But that makes sense since the exchange
# is closed.

In [7]:
# Calculate what the highest and lowest opening prices were for the stock in this period.
# A bit tricky, there are Nones for three open prices, 4/14, 4/17, and 5/1.
open_prices = [x.open for x in dd.values() if x.open is not None]
print(f"The highest opening price is {max(open_prices)}.")
print(f"The lowest opening price is {min(open_prices)}.")

The highest opening price is 53.11.
The lowest opening price is 34.0.


In [8]:
# What was the largest change in any one day (based on High and Low price)?
changes = [x.high - x.low for x in dd.values() if x.high is not None and x.low is not None]
print(f"The largest one-day change is {round(max(changes), 2)}.")

The largest one-day change is 2.81.


In [9]:
# What was the largest change between any two days (based on Closing Price)?
close_deltas = [(tuple(dd.values())[i].close, tuple(dd.values())[i+1].close) for i in range(len(dd)-1)]
# print([i for i in range(5)])
print(f"The largest change between close dates is {round(max([d[1] - d[0] for d in close_deltas]),2)}")

The largest change between close dates is 1.72


In [10]:
# What was the average daily trading volume during this year?
trade_vols = [x.traded_volume for x in dd.values()]
print(f"The average daily trading volume is {round(sum(trade_vols)/len(trade_vols),2)}")

The average daily trading volume is 89124.34


In [11]:
# What was the median trading volume during this year?
def median(nums):
    srt_nums = sorted(nums)
    len_nums = len(nums)
    if len(srt_nums) % 2 == 0:
        mid_point = int((len_nums / 2) - 1)
        return sum(srt_nums[mid_point:mid_point+2])/2
    elif len(srt_nums) > 0:
        return srt_nums[int((len_nums-1)/2)]
    else:
        return None

print(f"The median daily trading volume is {median(trade_vols)}.")

The median daily trading volume is 76286.0.
